In [29]:
!python -m spacy download ru_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB 93.1 kB/s eta 0:02:44
     ---------------------------------------- 0.0/15.3 MB 93.1 kB/s eta 0:02:44
     ---------------------------------------- 0.0/15.3 MB 93.1 kB/s eta 0:02:44
     --------------------------------------- 0.0/15.3 MB 122.9 kB/s eta 0:02:04
     --------------------------------------- 0.0/15.3 MB 122.9 kB/s eta 0:02:04
     --------------------------------------- 0.1/15.3 MB 208.4 kB/s eta 0:01:13
     --------------------------------------- 0.1/15.3 MB 288.1 kB/s eta 0:00:53
  


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy --gpu-id 0

[i] Saving to output directory: output
[i] Using GPU: 0

=========================== Initializing pipeline ===========================


Traceback (most recent call last):
  File "C:\Users\petya\AppData\Roaming\Python\Python310\site-packages\spacy\tokens\_serialize.py", line 228, in from_bytes
    msg = srsly.msgpack_loads(zlib.decompress(bytes_data))
zlib.error: Error -5 while decompressing data: incomplete or truncated stream

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\petya\AppData\Roaming\Python\Python310\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "C:\Users\petya\AppData\Roaming\Python\Python310\site-packages\spacy\cli\_util.py", line 92, in setup_cli
    command(prog_name=COMMAND)
  File "C:\Users\petya\AppData\Roaming\Python\Python310\site-packages\click\core.py", line 1157, in 

In [1]:
import spacy
import pandas as pd

from tqdm import tqdm
from spacy.tokens import DocBin
nlp = spacy.load("ru_core_news_sm")

In [2]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    # a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == 2:
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        # we need to set the (text)cat(egory) for each document
        #doc.cats["positive"] = label
        # put them into a nice list
        docs.append(doc)
    return docs

In [3]:
df = pd.read_csv('train.csv', encoding="utf8")
df_valid = pd.read_csv('valid.csv', encoding="utf8")
df = df.drop('Unnamed: 0',  axis=1)

df = df[df['sentiment'] != 0]
df_valid = df_valid[df_valid['sentiment']!=0]
df_valid = df_valid.drop('Unnamed: 0', axis=1)
print(df_valid.columns)
#take only one tenth of the whole dataset
df = df.sample(frac=0.2)
class_counts = df['sentiment'].value_counts()
#this is the measure against class disbalance
#in balanced_df we will recieve df with equality in quantity between classes
min_count = class_counts.min()
print(min_count)
balanced_df = df.groupby('sentiment').apply(lambda x: x.sample(n=min_count)).reset_index(drop=True)
#due to groupby balanced_df will be ordered, so we will shuffle dataframe
balanced_df = df.sample(frac=1).reset_index(drop=True)
print(df['sentiment'].value_counts())

train_data = [tuple((balanced_df.sample(frac=1)).iloc[i].values) for i in range((balanced_df.sample(frac=1)).shape[0])]
valid_data = [tuple(df_valid.iloc[i].values) for i in range(df_valid.shape[0])]



Index(['text', 'sentiment'], dtype='object')
9983
1    18130
2     9983
Name: sentiment, dtype: int64


In [4]:
# we are so far only interested in the first 5000 reviews
# this will keep the training time short.
# In practice take as much data as you can get.
# you can always reduce it to make the script even faster.

# first we need to transform all the training data
train_docs = make_docs(train_data)

100%|██████████| 28113/28113 [14:11<00:00, 33.02it/s]


In [5]:

# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:5000])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

100%|██████████| 5000/5000 [02:36<00:00, 31.98it/s]


на этом месте мы идем в https://spacy.io/usage/training#quickstart, там настраиваем под себя конфиг (textcat), копируем его руками(!) в base_config.cfg, указываем правильные пути до трейн и вэлид


In [9]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./valid.spacy --gpu-id 0

[i] Saving to output directory: output
[i] Using GPU: 0

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['textcat']
[i] Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       39.07    0.39
  0     200         52.40       42.10    0.42
  0     400         40.54       64.22    0.64
  0     600         36.29       69.91    0.70
  0     800         30.33       78.28    0.78
  0    1000         26.27       77.50    0.78
  0    1200         24.56       80.52    0.81
  0    1400         26.69       81.70    0.82
  0    1600         25.73       81.98    0.82
  0    1800         22.09       82.27    0.82
  0    2000         27.23       82.98    0.83
  0    2200         20.33       82.18    0.82
  0    2400         20.45       84.26    0.84
  0    2600    

In [12]:
import spacy
# load thebest model from training
nlp = spacy.load("output/model-best")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

type : ‘quit’ to exit
{'positive': 0.028318066149950027, 'negative': 0.9716819524765015}
the sentiment is negative
{'positive': 0.37664079666137695, 'negative': 0.623359203338623}
the sentiment is negative
